In [1]:
import sys
import os
sys.path.append('../')
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd

from utils.parser import *
from utils.helper import adult_target_value,adult_oneHot_names
from models.data_process import *
from models.run_MLP import MLP
from anchor import anchor_tabular

args = parse_args()
device = args.device

In [2]:
model = MLP().to(device)
ckpt = torch.load('../'+args.model_path+f'MPL_{args.epoch}.pth', map_location='cpu')
model.load_state_dict(ckpt)
model.eval()

torch.Size([6513, 25]) torch.int64


MLP(
  (net): Sequential(
    (0): Linear(in_features=25, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=2, bias=True)
  )
)

In [12]:
test_dataset = Adult_data(mode = 'test')
x_train = Adult_data(mode = 'train',tensor=False).dataset
sample_id = 1
x_test_idx,y_test_idx = test_dataset[sample_id]
pred = model.predict_single(x_test_idx)
print('Prediction: ',adult_target_value[pred],'\tlabel: ',pred )
x_test_idx = x_test_idx.detach().numpy()


torch.Size([6513, 25]) torch.int64
(26048, 25) int64
Prediction:  <=50K 	label:  0


In [27]:
explainer = anchor_tabular.AnchorTabularExplainer([b'<=50K', b'>50K'], adult_oneHot_names[:-1], x_train, categorical_names = {})
exp = explainer.explain_instance(x_test_idx, model.predict_detach, threshold=0.95)
print(f'Anchor sample_id: {sample_id}' ,exp.features())
print(f'Anchor sample_id: {sample_id}\n',exp.names())
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor sample_id: 1 [18, 23, 1, 2, 14, 5, 2, 0]
Anchor sample_id: 1
 ['race_Amer-Indian-Eskimo > -0.10', 'gender_Female > -0.70', 'educational-num <= -0.03', 'hours-per-week <= -0.03', 'occupation_Professional <= -0.38', 'workclass_Private > -1.52', 'age <= 0.69']
Precision: 0.82
Coverage: 0.00


In [28]:
res = []
for feature in exp.features():
    oneHot_names = adult_oneHot_names[feature].split('_')
    index = adult_column_names.index(oneHot_names[0])
    res.append(oneHot_names[0])
res = np.unique(res)
print(res)

['age' 'educational-num' 'gender' 'hours-per-week' 'occupation' 'race'
 'workclass']
